In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification, logging
import torch.nn as nn
from scipy.special import softmax
import pandas as pd
import numpy as np
import csv

In [ ]:
def SentimentAnalysis(texts):
  words = []

  tweet_proc = " ".join(texts)

  sentiment_labels = ['Negative', 'Neutral', 'Positive']

  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

  output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])

  #Showing only sentiment scores (Logits)
  scores = output[0][0].detach().numpy()

  #Transforming into probabilities
  scores = softmax(scores)

  negative_scores = []
  neutral_scores = []
  positive_scores = []

  #print(f'Tweet: {text}')
  for i in range(len(scores)):
    l = sentiment_labels[i]
    s = scores[i]
    #print(l, s)

    if l == 'Negative':
      negative_scores.append(s)
    elif l == 'Neutral':
      neutral_scores.append(s)
    elif l == 'Positive':
      positive_scores.append(s)


  #print("***********************************************")

  return negative_scores, neutral_scores, positive_scores

In [ ]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

df = pd.read_csv('SA_Category5.csv')
texts = df['0'].notnull()

sentiment_data = []

for i in texts:
    if not isinstance(i, str):
        i = str(i)

    negative_scores, neutral_scores, positive_scores = SentimentAnalysis(i)

    sentiment_data.append({
        "Sentence": i,
        "Negative Score": negative_scores,
        "Neutral Score": neutral_scores,
        "Positive Score": positive_scores
    })

output_df = pd.DataFrame(sentiment_data)

#Calculating the maximum score for each row
output_df['Max Score'] = output_df[['Negative Score', 'Neutral Score', 'Positive Score']].max(axis=1)

#Deciding the sentiment based on the maximum score
output_df['Sentiment'] = output_df.apply(lambda row: 'Negative' if row['Max Score'] == row['Negative Score']
                           else ('Neutral' if row['Max Score'] == row['Neutral Score'] else 'Positive'), axis=1)

df['Negative Score'] = output_df['Negative Score']
df['Neutral Score'] = output_df['Neutral Score']
df['Positive Score'] = output_df['Positive Score']
df['Max Score'] = output_df['Max Score']
df['Sentiment'] = output_df['Sentiment']

df.to_csv('SA_category_5.csv', index=False)
